In [12]:
import os
import json

import matplotlib.pyplot as plt
import cv2

import pandas as pd
import numpy as np

In [23]:
df = pd.DataFrame()
ori = pd.read_csv('/opt/ml/submissions/submission_swin_l_nms.csv') # original img inference result
tl = pd.read_csv('/opt/ml/submissions/submission_tl.csv') # top-left img inference result
tr = pd.read_csv('/opt/ml/submissions/submission_tr.csv') # top-right img inference result
bl = pd.read_csv('/opt/ml/submissions/submission_bl.csv') # bottom-left img inference result
br = pd.read_csv('/opt/ml/submissions/submission_br.csv') # bottom-right img inference result

df['PredictionString'] = tl['PredictionString']
df['image_id'] = [image_id.replace('test_tl','test') for image_id in tl['image_id']]
df

,PredictionString,image_id
0,0 0.14921635 467.17175 482.93326 512.0 511.614...,test/0000.jpg
1,0 0.33911347 134.77318 0.0 493.6403 260.00522 ...,test/0001.jpg
2,0 0.11744177 304.13574 321.75467 512.0 509.718...,test/0002.jpg
3,0 0.05046063 0.0 72.71631 220.2749 509.90366 0...,test/0003.jpg
4,NaN,test/0004.jpg
...,...,...
4866,0 0.7124018 319.57468 437.8221 509.19373 512.0...,test/4866.jpg
4867,9 0.044484623 0.17755635 234.7085 125.467896 4...,test/4867.jpg
4868,NaN,test/4868.jpg
4869,0 0.55517364 4.9404297 243.92058 512.0 509.898...,test/4869.jpg


In [24]:
for i in range(4871):
    predictions = []

    if type(ori['PredictionString'][i])==str:
        for j in range(len(ori['PredictionString'][i].split(' '))//6):
            o = ori['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            prediction = [id, score, xmin, ymin, xmax, ymax]
            predictions.append(prediction)

    if type(tl['PredictionString'][i])==str:
        for j in range(len(tl['PredictionString'][i].split(' '))//6):
            o = tl['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            prediction = [id, score, xmin, ymin, xmax, ymax]
            predictions.append(prediction)

    if type(tr['PredictionString'][i])==str:
        for j in range(len(tr['PredictionString'][i].split(' '))//6):
            o = tr['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            xmin += 512
            xmax += 512

            prediction = [id, score, xmin, ymin, xmax, ymax]
            predictions.append(prediction)

    if type(bl['PredictionString'][i])==str:
        for j in range(len(bl['PredictionString'][i].split(' '))//6):
            o = bl['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            ymin += 512
            ymax += 512

            prediction = [id, score, xmin, ymin, xmax, ymax]
            predictions.append(prediction)

    if type(br['PredictionString'][i])==str:
        for j in range(len(br['PredictionString'][i].split(' '))//6):
            o = br['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            xmin += 512
            xmax += 512
            ymin += 512
            ymax += 512

            prediction = [id, score, xmin, ymin, xmax, ymax]
            predictions.append(prediction)

    predictions.sort(key=lambda x:x[1])
    predictionstring = ''
    for pred in predictions:
        id, score, xmin, ymin, xmax, ymax = pred
        predictionstring += ' '.join([str(id), str(score), str(xmin), str(ymin), str(xmax), str(ymax)]) + ' '

    df['PredictionString'][i] = predictionstring

df

,PredictionString,image_id
0,7 0.0011880173 180 427 204 442 0 0.0014444184 ...,test/0000.jpg
1,2 0.0030085193 137 0 496 258 5 0.0043067615 76...,test/0001.jpg
2,3 0.0013745797000000001 773 225 867 238 0 0.00...,test/0002.jpg
3,5 0.0023976777 525 2 948 190 7 0.0029214097 14...,test/0003.jpg
4,5 0.0038108683999999995 204 514 511 775 0 0.00...,test/0004.jpg
...,...,...
4866,1 0.002080654 321 435 510 508 6 0.003413161299...,test/4866.jpg
4867,9 0.0029885855 522 322 770 508 7 0.00357476899...,test/4867.jpg
4868,9 0.0034535386 514 518 750 827 9 0.0035956435 ...,test/4868.jpg
4869,1 0.0022810152 512 30 986 512 7 0.0026890842 2...,test/4869.jpg


In [25]:
df.to_csv('/opt/ml/submissions/submission_tile_combined.csv')